## OpenParlamento Challenge

**Area di interesse**: Giurisprudenza

[Open Parlamento](https://parlamento17.openpolis.it), uno strumento di OpenPolis, contiene una quantità di informazioni e statistiche sull'operato dei nostri parlamentari:

![openpolis-89237](img/openpolis.png)

Qua troviamo un esempio per il Partito Democratico:  

https://parlamento17.openpolis.it/lista-dei-parlamentari-in-carica/camera/nome/asc/filter_group/71

L'obiettivo del progetto sarebbe replicare parte dei contenuti del sito in Jupyter. Per ottenere i dati, si può usare la comoda API fornita da OpenPolis all'indirizzo  [api3.openpolis.it](http://api3.openpolis.it)

Vediamo per esempio come ottenere i gruppi parlamentari: 

Notate che nella URL di sopra il `parlamento17` all'inizio indica la diciassettesima legislatura (la corrente nel 2018) e  il `71` alla fine, che è il codice assegnato al partito.  I codici li potete ottenere con le API di OpenPolis. Per vedere quale codice numerico è assegnato ai vari gruppi parlamentari, guardate qua (il `/17/` indica sempre la 17esima legislatura ): 

http://api3.openpolis.it/parlamento/17/groups

Se andate col browser a questa URL sopra, dovreste vedere qualcosa di simile - nell'interfaccia web potete comodamente navigare le api e vedere i risultati delle chiamate, che sono sempre in formato JSON: 

![openpolis-groups-23213](img/openpolis-groups.png)


* Per integrare dati con altre fonti, potreste scaricare delle pagine HTML da siti di news a tema politica, estrarre nomi di partiti politici e visualizzare in interfacce grafiche in Jupyter dei link a corrispondente pagine dei politici in OpenParlamento.  Notate che negli articoli i nomi dei partiti potrebbero essere leggermente diversi o sigle (es. 'PD','Lega' invece di 'Lega Nord'), quindi potreste implementare delle funzioni che cercano di identificare il partito.

* Come altri esempi, potreste mostrare i parlamentari del gruppo. Questa API prende i parlamentari del Partito Democratico (`group=71`) per la 17esima legislatura (`/17/`): 

http://api3.openpolis.it/parlamento/17/parliamentarians?group=71&page=1

Vengono ritornati solo 25 risultati. per avere i 25 successivi, incrementare il numero assegnato alla `page=1`


### Analisi attività parlamento: usare Wikidata per trovare i segretari di partito

Volendo, per trovare altre informazioni come i segretari di partito, potreste usare [Wikidata](https://www.wikidata.org). 

Provate ad eseguire questa query SPARQL (per eseguirla online cliccate il bottone blue play in basso a sinistra, i risultati appariranno sotto): http://tinyurl.com/y8yel2mv


Non li dà tutti, ma per es. almeno Renzi e Salvini li trovate. Anche se non conoscete il linguaggio SPARQL, se provate a smanettare volendo potreste anche ricavare qualcosa in più, prendendo come esempio attributi da [Partito Democratico](https://www.wikidata.org/wiki/Q47729) e [Renzi](https://www.wikidata.org/wiki/Q47563)

Il seguente codice Python lo potete ottenere cliccando nel query editor cliccando in basso a destra->Code -> Python

```python
#python3 -m pip install sparqlwrapper
#https://rdflib.github.io/sparqlwrapper/

from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""SELECT DISTINCT ?political_party ?political_partyLabel ?party_chief_representative ?party_chief_representativeLabel ?Openpolis_ID WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[it],it". }
  ?political_party wdt:P31 wd:Q7278.
  ?political_party wdt:P17 wd:Q38.
  OPTIONAL { ?political_party wdt:P210 ?party_chief_representative.
             ?party_chief_representative wdt:P1229 ?Openpolis_ID. }
}
LIMIT 500""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result)
```


Il codice Python vi stamperà qualcosa del genere: 

```
{'political_party': {'value': 'http://www.wikidata.org/entity/Q286140', 'type': 'uri'}, 'political_partyLabel': {'value': 'Sinistra Ecologia Libertà', 'xml:lang': 'it', 'type': 'literal'}}
{'political_party': {'value': 'http://www.wikidata.org/entity/Q541679', 'type': 'uri'}, 'political_partyLabel': {'value': 'Democratici di Sinistra', 'xml:lang': 'it', 'type': 'literal'}}
{'political_party': {'value': 'http://www.wikidata.org/entity/Q662849', 'type': 'uri'}, 'political_partyLabel': {'value': 'Alleanza Nazionale', 'xml:lang': 'it', 'type': 'literal'}}
{'political_party': {'value': 'http://www.wikidata.org/entity/Q764125', 'type': 'uri'}, 'political_partyLabel': {'value': "Partito d'Azione", 'xml:lang': 'it', 'type': 'literal'}}
```

In particolare per il Partito Democratico e Renzi, formattandolo bene otterresti questo:

```python
{
   'political_party': {
       'value': 'http://www.wikidata.org/entity/Q47729',
       'type': 'uri'
   },   
   'political_partyLabel': {
      'value': 'Partito Democratico',
      'xml:lang': 'it',
      'type': 'literal'
   },
   'party_chief_representative': {
       'value': 'http://www.wikidata.org/entity/Q47563',
       'type': 'uri'
   },         
   'party_chief_representativeLabel': {
     'value': 'Matteo Renzi',
     'xml:lang': 'it',
     'type': 'literal'
   },
   'Openpolis_ID': {
       'value': '6877',
       'type': 'literal'
   }
}
```


